In [4]:
import pandas as pd
# df.to_csv("Donnees_Nettoyées.csv")
df = pd.read_csv('/Users/josephpelham/Desktop/SISE/Python/Donnees/Donnees_Nettoyées(NA supprimé pour classification).csv')

In [5]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [6]:
# Dummies pour 'Nature_mutation' et'Nature_culture'
df_dummies = pd.get_dummies(df, columns=['Nature_mutation','Nature_culture','Code_departement'],dtype=float )

In [7]:
# Replace missing values in Nombre_pieces_principales by the corresponding median in Type_Local

result = df_dummies.groupby(['Type_local'])['Nombre_pieces_principales'].median().reset_index(name='med')
median_mapping = result.set_index('Type_local')['med'].to_dict()
df_dummies['Nombre_pieces_principales'] = df_dummies['Nombre_pieces_principales'].fillna(df_dummies['Type_local'].map(median_mapping))

In [8]:
# Replace missing values in Nombre_pieces_principales by the median in Type_Local
res = df_dummies.groupby(['Type_local'])['Surface_reelle_bati'].median().reset_index(name='med')
median_mapping = res.set_index('Type_local')['med'].to_dict()
df_dummies['Surface_reelle_bati'] = df_dummies['Surface_reelle_bati'].fillna(df_dummies['Type_local'].map(median_mapping))

In [9]:
# Regarder les NA
pd.set_option('display.max_rows', None)

# % of NA 
df_dummies.isna().sum()

Unnamed: 0                                                 0
No_disposition                                             0
Date_mutation                                              0
Valeur_fonciere                                          280
No_voie                                                 3689
Type_de_voie                                           19822
Code_voie                                                  0
Voie                                                       0
Code_postal                                               11
Commune                                                    0
Code_commune                                               0
Section                                                   13
No_plan                                                    0
1er_lot                                               114080
Nombre_de_lots                                             0
Code_type_local                                            0
Type_local              

In [10]:
# Reset the display option to the default
pd.reset_option('display.max_rows')

In [11]:
# Creer un subset en exlucant toutes les colonnes qu'on n'utilise pas pour la classification 
subset = (df_dummies.loc[:, ~df_dummies.columns.isin(['No_disposition', 'Date_mutation', 'Valeur_fonciere',
                                                            'No_voie', 'Type_de_voie', 'Voie','Code_postal',
                                                            'Commune', 'Code_commune', 'Section','No_plan',
                                                            '1er_lot', 'Nombre_de_lots', 'Code_type_local','Type_local',
                                                            'Surface_terrain', 'adresse', 'year','Month','Quarter', 'Quarter_Label',
                                                     'Code_voie', 'month_year', 'Year'])])
# Creer la liste des variables independantes pour la classification 
feature_cols = subset.columns.tolist()
feature_cols = feature_cols[1:]



In [12]:
# Preparer la classification              
X = df_dummies[feature_cols] # Features
y = df_dummies.Code_type_local # Target variable

In [13]:
# Split data
from sklearn import tree

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [14]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(max_depth = 3, random_state=0)

In [15]:
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

In [16]:
#Predict the response for test dataset
y_pred = clf.predict(X_test)
y_pred

array([1., 1., 3., ..., 3., 4., 2.])

In [17]:
accuracy_score = metrics.accuracy_score(y_test, y_pred)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9121672888403289


In [18]:
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(y_test,y_pred),
                  columns=['pred_1','pred_2','pred_3','pred_4'],
                  index=['obs_1','obs_2','obs_3','obs_4'])

mc

,pred_1,pred_2,pred_3,pred_4
obs_1,24882,4496,0,40
obs_2,1118,17264,0,33
obs_3,0,0,14431,0
obs_4,0,0,101,3533


In [20]:
########### Random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

In [21]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [22]:
y_pred = rf.predict(X_test)

In [23]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9489817596892167


In [24]:
pd.crosstab(y_test,y_pred, colnames=['pred'], rownames=['obs'], margins=True)

pred,1.0,2.0,3.0,4.0,All
obs,,,,,
1.0,27736,1644,0,38,29418
2.0,1524,16866,0,25,18415
3.0,0,0,14424,7,14431
4.0,14,10,100,3510,3634
All,29274,18520,14524,3580,65898


In [25]:
from sklearn.metrics import accuracy_score, recall_score, f1_score

# Calculate accuracy 
#Accuracy = (Number of Correct Predictions) / (Total Number of Predictions)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy {accuracy}")

# Calculate recall for each class and get a list of recall scores 
# Recall (Class i) = (True Positives for Class i) / (Total Actual Positives for Class i)
recall_scores = recall_score(y_test, y_pred, average=None)
print(f"Recall_scores {recall_scores}")

# Calculate F1 score for each class and get a list of F1 scores
f1_scores = f1_score(y_test, y_pred, average=None)
print(f"f1_scores {f1_scores}")
weighted_f1 = f1_score(y_test, y_pred, average='weighted')
print(f"weighted f1_score {weighted_f1}")


Accuracy 0.9489817596892167
Recall_scores [0.94282412 0.91588379 0.99951493 0.96587782]
f1_scores [0.94513733 0.91328009 0.99630461 0.97310785]
weighted f1_score 0.9489824946196409
